In [12]:
import pandas as pd
import numpy as np
import joblib

# -------------------------
# PATHS
# -------------------------
MODEL_PATH = "demand_model.pkl"
TEST_PATH = "test_processed.csv"
OUT_PATH = "test_predictions_with_risk.csv"

# -------------------------
# LOAD MODEL & DATA
# -------------------------
model = joblib.load(MODEL_PATH)
test = pd.read_csv(TEST_PATH)

# -------------------------
# FEATURES (MUST MATCH TRAINING)
# -------------------------
FEATURES = [
    "day_of_week",
    "is_weekend",
    "month",
    "lag_1",
    "lag_7",
    "lag_14",
    "rolling_mean_7",
    "rolling_mean_14",
    "inventory_level"
]

# -------------------------
# PREDICTION
# -------------------------
test["predicted_sales"] = model.predict(test[FEATURES])

test["predicted_sales"] = test["predicted_sales"].clip(lower=0).round(0)

# -------------------------
# RISK LOGIC
# -------------------------
test["stockout_risk"] = np.where(
    test["predicted_sales"] > test["inventory_level"],
    "HIGH",
    "LOW"
)

# -------------------------
# FINAL OUTPUT
# -------------------------
final = test[[
    "id",
    "predicted_sales",
    "inventory_level",
    "stockout_risk"
]]

# -------------------------
# SAVE
# -------------------------
final.to_csv(OUT_PATH, index=False)

print("✅ Test prediction & risk analysis complete")
print(final.head())




✅ Test prediction & risk analysis complete
   id  predicted_sales  inventory_level stockout_risk
0   0             13.0              134           LOW
1   1             15.0              134           LOW
2   2             15.0              134           LOW
3   3             16.0              134           LOW
4   4             17.0              134           LOW
